# Homework 2: training pipeline

This code will test your homework 2 solutions by using them in a complete ML pipeline. You should run this code in order to tune your model and save your model weights (which will also be uploaded as part of your solution)

In [1]:
# Download the training data from the homework2 folder:
# unzip using tar xzvvf nsynth_subset.tar.gz
# (this is a small subset of the "nsynth" dataset: https://magenta.tensorflow.org/datasets/nsynth)

In [30]:
import homework2

### Install and Load Required Libraries  

In [3]:
# !pip install librosa
# !pip install torch
# !pip install glob
# !pip install numpy

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import random
import glob

In [32]:
BATCH_SIZE = 16
torch.use_deterministic_algorithms(True)

In [33]:
if not len(homework2.audio_paths):
    print("You probably need to set the dataroot folder correctly")

In [34]:
# Some helper functions. These are the same as what the autograder runs.

In [35]:
# Split dataset into train / valid / test
def split_data(waveforms, labels, train_ratio=0.7, valid_ratio=0.15):
    assert(train_ratio + valid_ratio < 1)
    test_ratio = 1 - (train_ratio + valid_ratio)
    N = len(waveforms)
    Ntrain = int(N * train_ratio)
    Nvalid = int(N * valid_ratio)
    Ntest = int(N * test_ratio)
    Wtrain = waveforms[:Ntrain]
    Wvalid = waveforms[Ntrain:Ntrain + Nvalid]
    Wtest = waveforms[Ntrain + Nvalid:]
    ytrain = labels[:Ntrain]
    yvalid = labels[Ntrain:Ntrain + Nvalid]
    ytest = labels[Ntrain + Nvalid:]
    return Wtrain,Wvalid,Wtest,ytrain,yvalid,ytest

In [36]:
def process_data(W, feature_function):
    return [feature_function(path) for path in W]

In [37]:
class InstrumentDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.features[idx]
        label = self.labels[idx]

        return features, torch.tensor(label, dtype=torch.long)

In [38]:
class Loaders():
    def __init__(self, waveforms, labels, feature_function, seed = 0):
        torch.manual_seed(seed)
        random.seed(seed)
        self.Wtrain, self.Wvalid, self.Wtest, self.ytrain, self.yvalid, self.ytest = split_data(waveforms, labels)
        
        self.Xtrain = process_data(self.Wtrain, feature_function)
        self.Xvalid = process_data(self.Wvalid, feature_function)
        self.Xtest = process_data(self.Wtest, feature_function)
        
        self.dataTrain = InstrumentDataset(self.Xtrain, self.ytrain)
        self.dataValid = InstrumentDataset(self.Xvalid, self.yvalid)
        self.dataTest = InstrumentDataset(self.Xtest, self.ytest)
        
        self.loaderTrain = DataLoader(self.dataTrain, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
        self.loaderValid = DataLoader(self.dataValid, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderTest = DataLoader(self.dataTest, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [39]:
class Pipeline():
    def __init__(self, module, learning_rate, seed = 0):
        # These two lines will (mostly) make things deterministic.
        # You're welcome to modify them to try to get a better solution.
        torch.manual_seed(seed)
        random.seed(seed)

        self.device = torch.device("cpu") # Can change this if you have a GPU, but the autograder will use CPU
        self.criterion = nn.CrossEntropyLoss()
        
        self.model = module.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def evaluate(self, loader, which = "valid"):
        self.model.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                #loss = criterion(outputs, labels) # validation loss

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = correct / total
        
        return acc
    
    def train(self, loaders,
          num_epochs=1, # Train for a single epoch by default
          model_path=None): # (Optionally) provide a path to save the best model
        val_acc = 0
        best_val_acc = 0
        for epoch in range(num_epochs):
            self.model.train()
            
            losses = []

            for inputs, labels in loaders.loaderTrain:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                losses.append(float(loss))
            
            self.model.eval()
            val_acc = self.evaluate(loaders.loaderValid)
            print("Epoch " + str(epoch) + ", loss = " + str(sum(losses)/len(losses)) +\
                  ", validation accuracy = " + str(val_acc))

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                if (model_path):
                    torch.save(self.model.state_dict(), model_path)
        print("Final validation accuracy = " + str(val_acc) + ", best = " + str(best_val_acc))
        return val_acc, best_val_acc

    def load(self, path):
        self.model.load_state_dict(torch.load(path, weights_only=True))

In [13]:
# The function below is the basis of how the autograder tests your code. Try to understand this one.

In [40]:
def test(waveforms, labels, feature_func, classifier, learning_rate, path):
    print("Extracting features...")
    test_loaders = Loaders(waveforms, labels, feature_func)
    test_pipeline = Pipeline(classifier, learning_rate)
    
    # Note: the autograder will not run this line: it will just load your saved model (next line)
    acc, best_acc = test_pipeline.train(test_loaders, 40, path)
    
    test_pipeline.load(path)
    test_acc = test_pipeline.evaluate(test_loaders.loaderTest)
    print("Test accuracy = " + str(test_acc))

In [15]:
# 1. Paths, labels, waveforms

In [41]:
# Once you've written the corresponding code in homework2.py, print these out or visualize them if you want
homework2.waveforms
homework2.labels

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,


In [17]:
# 2. MFCC

In [18]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mfcc,
     homework2.MLPClassifier(),
     0.0001,
     "best_mlp_model.weights")

Extracting features...
Epoch 0, loss = 1.5908317118883133, validation accuracy = 0.7560975609756098
Epoch 1, loss = 0.7122069688306915, validation accuracy = 0.8617886178861789
Epoch 2, loss = 0.4246127920018302, validation accuracy = 0.9186991869918699
Epoch 3, loss = 0.2533628450085719, validation accuracy = 0.943089430894309
Epoch 4, loss = 0.15960726980119944, validation accuracy = 0.983739837398374
Epoch 5, loss = 0.11656166840758589, validation accuracy = 0.975609756097561
Epoch 6, loss = 0.09354176020456685, validation accuracy = 0.983739837398374
Epoch 7, loss = 0.07725009839567873, validation accuracy = 0.983739837398374
Epoch 8, loss = 0.06428412291117841, validation accuracy = 0.983739837398374
Epoch 9, loss = 0.05437703301302261, validation accuracy = 0.983739837398374
Epoch 10, loss = 0.04740078416135576, validation accuracy = 0.983739837398374
Epoch 11, loss = 0.0406210839541422, validation accuracy = 0.983739837398374
Epoch 12, loss = 0.03551760531910178, validation accu

In [19]:
# 3. Spectrogram

In [20]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_spec,
     homework2.SimpleCNN(),
     0.0001,
     "best_spec_model.weights")

Extracting features...
Epoch 0, loss = 0.6035331885019938, validation accuracy = 0.9105691056910569
Epoch 1, loss = 0.538013911081685, validation accuracy = 0.9186991869918699
Epoch 2, loss = 0.5128232116500536, validation accuracy = 0.9186991869918699
Epoch 3, loss = 0.4902407286895646, validation accuracy = 0.9349593495934959
Epoch 4, loss = 0.4596041101548407, validation accuracy = 0.943089430894309
Epoch 5, loss = 0.4314737444122632, validation accuracy = 0.967479674796748
Epoch 6, loss = 0.39708611120780307, validation accuracy = 0.983739837398374
Epoch 7, loss = 0.3630213008986579, validation accuracy = 0.983739837398374
Epoch 8, loss = 0.3344951445857684, validation accuracy = 0.975609756097561
Epoch 9, loss = 0.31520238146185875, validation accuracy = 0.967479674796748
Epoch 10, loss = 0.293820855104261, validation accuracy = 0.975609756097561
Epoch 11, loss = 0.27234212433298427, validation accuracy = 0.975609756097561
Epoch 12, loss = 0.26835182971424526, validation accuracy 

In [21]:
# 4. Mel-spectrogram

In [22]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mel,
     homework2.SimpleCNN(),
     0.0001,
     "best_mel_model.weights")

Extracting features...
Epoch 0, loss = 0.44188548541731304, validation accuracy = 0.7723577235772358
Epoch 1, loss = 0.30361509944001835, validation accuracy = 0.9512195121951219
Epoch 2, loss = 0.24448492667741245, validation accuracy = 0.991869918699187
Epoch 3, loss = 0.21370177736712825, validation accuracy = 0.991869918699187
Epoch 4, loss = 0.17559397406876087, validation accuracy = 0.991869918699187
Epoch 5, loss = 0.15761136263608932, validation accuracy = 0.991869918699187
Epoch 6, loss = 0.15048977732658386, validation accuracy = 0.991869918699187
Epoch 7, loss = 0.12523397679130235, validation accuracy = 0.991869918699187
Epoch 8, loss = 0.11561188701954153, validation accuracy = 1.0
Epoch 9, loss = 0.10735310448540582, validation accuracy = 1.0
Epoch 10, loss = 0.09551818254921171, validation accuracy = 0.991869918699187
Epoch 11, loss = 0.08560910469128026, validation accuracy = 1.0
Epoch 12, loss = 0.09295604812602203, validation accuracy = 1.0
Epoch 13, loss = 0.08118230

In [23]:
# 5. Constant-Q

In [24]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_q_model.weights")

Extracting features...
Epoch 0, loss = 0.5641960087749693, validation accuracy = 0.7235772357723578
Epoch 1, loss = 0.48745852212111157, validation accuracy = 0.8048780487804879
Epoch 2, loss = 0.42389339788092506, validation accuracy = 0.8617886178861789
Epoch 3, loss = 0.37617990747094154, validation accuracy = 0.9024390243902439
Epoch 4, loss = 0.33216064588891137, validation accuracy = 0.9186991869918699
Epoch 5, loss = 0.3060038710633914, validation accuracy = 0.943089430894309
Epoch 6, loss = 0.27534177485439515, validation accuracy = 0.959349593495935
Epoch 7, loss = 0.24883189590440857, validation accuracy = 0.959349593495935
Epoch 8, loss = 0.2259158289266957, validation accuracy = 0.975609756097561
Epoch 9, loss = 0.20527227140135235, validation accuracy = 0.975609756097561
Epoch 10, loss = 0.1918356849087609, validation accuracy = 0.975609756097561
Epoch 11, loss = 0.17276435076362556, validation accuracy = 0.975609756097561
Epoch 12, loss = 0.1713687042809195, validation ac

In [25]:
# 6. Pitch shift

In [26]:
test(homework2.augmented_waveforms,
     homework2.augmented_labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_augmented_model.weights")

Extracting features...
Epoch 0, loss = 0.4899570569396019, validation accuracy = 0.907859078590786
Epoch 1, loss = 0.3125536496992464, validation accuracy = 0.948509485094851
Epoch 2, loss = 0.24569645179090677, validation accuracy = 0.975609756097561
Epoch 3, loss = 0.1970778692651678, validation accuracy = 0.9701897018970189
Epoch 4, loss = 0.1636591579903055, validation accuracy = 0.986449864498645
Epoch 5, loss = 0.14767476233343282, validation accuracy = 0.991869918699187
Epoch 6, loss = 0.11875775980728644, validation accuracy = 0.994579945799458
Epoch 7, loss = 0.10115399473795185, validation accuracy = 0.994579945799458
Epoch 8, loss = 0.0946985294835435, validation accuracy = 0.994579945799458
Epoch 9, loss = 0.07925940599913399, validation accuracy = 1.0
Epoch 10, loss = 0.06999657556621565, validation accuracy = 1.0
Epoch 11, loss = 0.06725592271390336, validation accuracy = 0.997289972899729
Epoch 12, loss = 0.05064904115473231, validation accuracy = 0.997289972899729
Epoch

In [27]:
# 7. Extend your model to handle four classes and creatively improve its performance

In [43]:
test(homework2.waveforms,
     homework2.labels_7,
     homework2.feature_func_7,
     homework2.model_7,
     0.001,
     "best_model_7.weights")

Extracting features...
Epoch 0, loss = 0.4241877223054568, validation accuracy = 0.8617886178861789
Epoch 1, loss = 0.3440834962659412, validation accuracy = 0.8536585365853658
Epoch 2, loss = 0.33160771967636216, validation accuracy = 0.8536585365853658
Epoch 3, loss = 0.31090428349044585, validation accuracy = 0.6585365853658537
Epoch 4, loss = 0.28889471830593216, validation accuracy = 0.8536585365853658
Epoch 5, loss = 0.27447719582253033, validation accuracy = 0.8699186991869918
Epoch 6, loss = 0.28870217150284183, validation accuracy = 0.7642276422764228
Epoch 7, loss = 0.2653372469875548, validation accuracy = 0.8536585365853658
Epoch 8, loss = 0.2603862887869279, validation accuracy = 0.8617886178861789
Epoch 9, loss = 0.23998725533278453, validation accuracy = 0.9349593495934959
Epoch 10, loss = 0.22942330067356428, validation accuracy = 0.9024390243902439
Epoch 11, loss = 0.2089009073873361, validation accuracy = 0.8780487804878049
Epoch 12, loss = 0.21071116315821806, valida